In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
key = os.getenv("OPENAI_API_KEY")

In [13]:
llm =  ChatOpenAI(openai_api_key=key, model_name="gpt-3.5-turbo", temperature=0.5)

C:\Users\Asus\AppData\Local\Temp\ipykernel_22584\4154318361.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm =  ChatOpenAI(openai_api_key=key, model_name="gpt-3.5-turbo", temperature=0.5)


In [15]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [17]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [16]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [19]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [24]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [21]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [22]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [23]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [25]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [28]:
file_path = r"D:\Projects\mcqgen\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()
print(TEXT)

Ninja's 8-inch Chef’s Knife is an all-around great knife. The rounded handle is comfortable, and the knife is well-balanced, making it easy to handle. The stainless steel blade is tough and holds an edge well. I've been using this knife daily for two months and felt the need to sharpen it only twice. There are more-expensive knives in this guide that are better in some ways, but for most people this is the best combination of quality and price.

Take the money you save buying this and invest in a good sharpener. Ninja sells this knife with a sharpener for $90 (separately, the sharpener is $50, so you save a little), but I haven't tested it. If you're a beginner, I suggest the simplicity of a two-stage electric sharpener like the Presto EverSharp ($45 at Amazon). If you know how to sharpen with a stone, I like Shapton’s medium grit (1,000) ceramic stone ($50 at Amazon). If you've never used a stone, fear not; it's not that difficult. YouTube is full of videos to get you started, but I r

In [29]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [30]:
NUMBER=5 
SUBJECT="cooking"
TONE="simple"

In [31]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Asus\AppData\Local\Temp\ipykernel_22584\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Ninja's 8-inch Chef’s Knife is an all-around great knife. The rounded handle is comfortable, and the knife is well-balanced, making it easy to handle. The stainless steel blade is tough and holds an edge well. I've been using this knife daily for two months and felt the need to sharpen it only twice. There are more-expensive knives in this guide that are better in some ways, but for most people this is the best combination of quality and price.

Take the money you save buying this and invest in a good sharpener. Ninja sells this knife with a sharpener for $90 (separately, the sharpener is $50, so you save a little), but I haven't tested it. If you're a beginner, I suggest the simplicity of a two-stage electric sharpener like the Presto EverSharp ($45 at Amazon). If you know how to sharpen with a stone, I like Shapton’s medium grit (1,000) ceramic stone ($50 at Amazon). If you've never used a stone, fear not; it's not that difficult. YouTube is full of vid

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:Ninja's 8-inch Chef’s Knife is an all-around great knife. The rounded handle is comfortable, and the knife is well-balanced, making it easy to handle. The stainless steel blade is tough and holds an edge well. I've been using this knife daily for two months and felt the need to sharpen it only twice. There are more-expensive knives in this guide that are better in some ways, but for most people this is the best combination of quality and price.

Take the money you save buying this and invest in a good sharpener. Ninja sells this knife with a sharpener for $90 (separately, the sharpener is $50, so you save a little), but I haven't tested it. If you're a beginner, I suggest the simplicity of a two-stage electric sharpener like the Presto EverSharp ($45 at Amazon). If you know how to sharpen with a stone, I like Shapton’s medium grit (1,000) ceramic stone ($50 at Amazon). If you've never used a stone, fear not; it's not that difficult. You

In [40]:
quiz = response.get("quiz")

In [44]:
quiz = json.loads(quiz)

In [45]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
            ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct": correct})

In [46]:
quiz_table_data

[{'MCQ': "What is the material of the Ninja's 8-inch Chef’s Knife blade?",
  'Choices': 'a:Plastic b:Stainless steel c:Ceramic d:Titanium',
  'Correct': 'b'},
 {'MCQ': "How often did the reviewer need to sharpen the Ninja's Chef’s Knife after two months of daily use?",
  'Choices': 'a:Once a week b:Twice c:Every day d:Never',
  'Correct': 'b'},
 {'MCQ': 'What is a recommended electric sharpener for beginners mentioned in the text?',
  'Choices': "a:Shapton’s ceramic stone b:Presto EverSharp c:Ninja sharpener d:Victorinox's Fibrox Pro",
  'Correct': 'b'},
 {'MCQ': "What is the price of the Victorinox's Fibrox Pro knife mentioned in the text?",
  'Choices': 'a:$45 b:$50 c:$55 d:$90',
  'Correct': 'c'},
 {'MCQ': "What feature of the Victorinox's Fibrox Pro knife makes it stand out according to the text?",
  'Choices': 'a:High-quality construction b:Nonstick finish c:Comfortable handle d:Expensive price',
  'Correct': 'b'}]

In [48]:
quiz = pd.DataFrame(quiz_table_data)

In [49]:
quiz.to_csv("knifequiz.csv", index=False)